In [1]:
import sympy as sym
import numpy as np
import plotly.graph_objects as go

def choice_risk(xmin, xmax, alpha1=0.5, gamma2=0.75):
    xstart = xmin
    xend = xmax
    interval = 100
    c = sym.symbols('c')
    α = alpha1
    γ = gamma2
    # functions altered based on textbook defn 
    U1 = 1-((sym.exp(-α*c))/α)
    U2 = (c**(1-γ))/(1-γ)
    U3 = sym.log(c)
 
    U1_x = sym.lambdify(c, U1, modules=["numpy"])
    U2_x = sym.lambdify(c, U2, modules=["numpy"])
    U3_x = sym.lambdify(c, U3, modules=["numpy"])
    
    x = np.linspace(xstart,xend,interval)

    trace_U1 = go.Scatter(x = x, y = U1_x(x), name='CARA α={}'.format(α))
    trace_U2 = go.Scatter(x = x, y = U2_x(x), name='CRRA γ={}'.format(γ))
    trace_U3 = go.Scatter(x = x, y = U3_x(x), name='CRRA γ=1')
    print(trace_U3)
    risk_layout = go.Layout(title = "Expected Utility & Risk Aversion",
                           xaxis = dict(title="Consumption/Wealth"),
                           yaxis = dict(title="Value of Utility U(c), V(c)"),
                           margin = dict(l=50, r=50, b=50, t=50),
                           height=750, width=750)
    
    fig = go.Figure(data=[trace_U1,trace_U2,trace_U3],layout=risk_layout)
    
    return fig


ModuleNotFoundError: No module named 'sympy'

In [ ]:
choice_risk(0,20,0.5,0.75).show()

Scatter({
    'name': 'CRRA γ=1',
    'x': array([ 0.        ,  0.2020202 ,  0.4040404 ,  0.60606061,  0.80808081,
                 1.01010101,  1.21212121,  1.41414141,  1.61616162,  1.81818182,
                 2.02020202,  2.22222222,  2.42424242,  2.62626263,  2.82828283,
                 3.03030303,  3.23232323,  3.43434343,  3.63636364,  3.83838384,
                 4.04040404,  4.24242424,  4.44444444,  4.64646465,  4.84848485,
                 5.05050505,  5.25252525,  5.45454545,  5.65656566,  5.85858586,
                 6.06060606,  6.26262626,  6.46464646,  6.66666667,  6.86868687,
                 7.07070707,  7.27272727,  7.47474747,  7.67676768,  7.87878788,
                 8.08080808,  8.28282828,  8.48484848,  8.68686869,  8.88888889,
                 9.09090909,  9.29292929,  9.49494949,  9.6969697 ,  9.8989899 ,
                10.1010101 , 10.3030303 , 10.50505051, 10.70707071, 10.90909091,
                11.11111111, 11.31313131, 11.51515152, 11.71717172, 11.9191

In [ ]:
import numpy as np
import plotly.graph_objects as go
import numpy as np
import sympy as sym
import pynverse as pyn

def choice_risk(prf,alpha1=0.5,alpha2=0.75,gamma1=0.5,gamma2=0.75,bpay=1,gpay=3,bprob=0.5,gprob=0.5):
    interval = 50
    c = sym.symbols('c')
    preference = prf
    α1 = alpha1
    α2 = alpha2
    γ1 = gamma1
    γ2 = gamma2
    # Utility functions: return an array containing 3D Utility and 
    def CARA():
        U1 = 1-(sym.exp(-α1*c))
        U2 = 1-(sym.exp(-α2*c))
        return U1, U2 
    def CRRA():
        U1 = (c**(1-γ1)-1)/(1-γ1)
        U2 = (c**(1-γ2)-1)/(1-γ2)
        return U1, U2 
    
    # Dispatcher aides in calling different functions based on parameters.    
    dispatcher = {'CARA': CARA, 
                  'CRRA': CRRA,}
    
    U1,U2 = dispatcher[preference]() # Assign return value of different function calls.

    U1_x = sym.lambdify(c, U1, modules=["numpy"])
    U2_x = sym.lambdify(c, U2, modules=["numpy"])
    
    # Gamble and its outcomes
    bad_pay = bpay
    good_pay = gpay
    bad_prob = bprob
    good_prob = gprob
    
    xvals = np.linspace(bad_pay/10,good_pay*2,interval)

    EX = (bad_prob*bad_pay + good_prob*good_pay) # Expected payoff E(X) of the Gamble. E(X) is same for 1&2.
    EU1 = (bad_prob*U1_x(bad_pay) + good_prob*U1_x(good_pay)) # Expected Utility E(U) for Gambler 1.
    EU2 = (bad_prob*U2_x(bad_pay) + good_prob*U2_x(good_pay)) # Expected Utility E(U) for Gambler 2.
    
    CE1 = pyn.inversefunc(U1_x, y_values=EU1) # Find the Certainty Equivalent for Gambler 1
    CE2 = pyn.inversefunc(U2_x, y_values=EU2) # Find the Certainty Equivalent for Gambler 2
    
    RP1 = EX - CE1 # Risk Premium for Gambler 1
    RP2 = EX - CE2 # Risk Premium for Gambler 2
    
    # Draw Utility curves for gamblers 1 & 2
    trace_U1 = go.Scatter(x = xvals, y = U1_x(xvals), name='U1: Gambler 1')
    trace_U2 = go.Scatter(x = xvals, y = U2_x(xvals), name='U2: Gambler 2')
    risk_layout = go.Layout(title = "Expected Utility & Risk Aversion",
                           xaxis = dict(title="Consumption/Wealth"),
                           yaxis = dict(title="Value of Utility U(c), V(c)"),
                           margin = dict(l=50, r=50, b=50, t=50),
                           height=750, width=1000)
    
    fig = go.Figure(data=[trace_U1,trace_U2],layout=risk_layout)
    
    # Add Measurements for Gambler 1
    fig.add_shape(
            # Expected Utility line for U1
            dict(type="line", x0=bad_pay, y0=U1_x(bad_pay),
                 x1=good_pay, y1=U1_x(good_pay), line=dict(color="Blue",width=3, dash="dash")))  
    fig.add_trace(
            go.Scatter(
            x=[bad_pay, CE1, EX, EX, good_pay],
            y=[U1_x(bad_pay), 0,0, EU1, U1_x(good_pay)],
            mode="markers+text",
            text=["Utility(BadState)","Certainty Equivalent: {0:.2f}<br> Gambler 1".format(CE1), 
                  "Exp. Value of Gamble: {0:.2f}".format(EX), 
                  "Exp.Utility<br>Gambler 1","Utility(GoodState)"],
            textposition=["bottom center","top right","bottom right","bottom right","bottom right"],
            textfont=dict(family="Courier New, monospace",size=8,color="Blue"), showlegend=False))
    fig.add_shape(
            # Risk Premium line
            dict(type="line", x0=EX, y0=EU1,
                    x1= CE1, y1=EU1, line=dict(color="Blue",width=2,)))
    fig.add_shape(
             # Vertical line to Cert.Equiv
            dict(type="line", x0=CE1, y0=0,
                    x1= CE1, y1=EU1, line=dict(color="Blue",width=2,)))        
    fig.add_annotation(
            x=(EX+CE1)/2,y=(EU1),
            xref="x",yref="y",
            text="Gambler 1:<br> Risk Premium: {0:.2f}".format(RP1),
            showarrow=True,
            font=dict(family="Courier New, monospace",size=10,color="#ffffff"),
            align="center",
            arrowhead=2,arrowsize=1,arrowwidth=2,arrowcolor="#636363",
            ax=100,ay=-50,
            bordercolor="#c7c7c7",borderwidth=2,borderpad=4,
            bgcolor="Blue",opacity=0.8)    
    
   # Add Measurements for Gambler 2 
    fig.add_shape(
            # Expected Utility line for U2
            dict(type="line", x0=bad_pay, y0=U2_x(bad_pay),
                 x1=good_pay, y1=U2_x(good_pay), line=dict(color="Red",width=3, dash="dash")))  
    fig.add_trace(
            go.Scatter(
            x=[bad_pay, CE2, EX, good_pay],
            y=[U2_x(bad_pay),0, EU2, U2_x(good_pay)],
            mode="markers+text",
            text=["Utility(BadState)","Certainty Equivalent: {0:.2f}<br> Gambler 2".format(CE2), 
                  "Exp.Utility<br>Gambler 2","Utility(GoodState)"],
            textposition=["bottom left","bottom left","bottom right","bottom right"],
            textfont=dict(family="Courier New, monospace",size=8,color="Blue"), showlegend=False))
    fig.add_shape(
            # Risk Premium line
            dict(type="line", x0=EX, y0=EU2,
                    x1= CE2, y1=EU2, line=dict(color="Red",width=2,)))
    fig.add_shape(
             # Vertical line to Cert.Equiv
            dict(type="line", x0=CE2, y0=0,
                    x1= CE2, y1=EU2, line=dict(color="Red",width=2,)))        
    fig.add_annotation(
            x=(EX+CE2)/2,y=(EU2),
            xref="x",yref="y",
            text="Gambler 2:<br> Risk Premium: {0:.2f}".format(RP2),
            showarrow=True,
            font=dict(family="Courier New, monospace",size=10,color="#ffffff"),
            align="center",
            arrowhead=2,arrowsize=1,arrowwidth=2,arrowcolor="#636363",
            ax=100,ay=-75,
            bordercolor="#c7c7c7",borderwidth=2,borderpad=4,
            bgcolor="Red",opacity=0.8)
    
    return fig

In [ ]:
choice_risk('CARA',1,2,0.25,0.75,1,3,0.5,0.5).show()